<a href="https://colab.research.google.com/github/MohitDhungana/duplicate_question_detection/blob/master/predictedReviewdTest.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
import numpy as np

import re

from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
def text_to_word_list(text):
#     gets a whole question inside text variable on which  preprocessing is done and then the question is splitted into word indices and returned
    text = str(text)
    text = text.lower()

    # Clean the text
    text = re.sub(r"[^A-Za-z0-9^,!.\/'+-=]", " ", text)
    text = re.sub(r"what's", "what is ", text)
    text = re.sub(r"\'s", " ", text)
    text = re.sub(r"\'ve", " have ", text)
    text = re.sub(r"can't", "cannot ", text)
    text = re.sub(r"n't", " not ", text)
    text = re.sub(r"i'm", "i am ", text)
    text = re.sub(r"\'re", " are ", text)
    text = re.sub(r"\'d", " would ", text)
    text = re.sub(r"\'ll", " will ", text)
    text = re.sub(r",", " ", text)
    text = re.sub(r"\.", " ", text)
    text = re.sub(r"!", " ! ", text)
    text = re.sub(r"\/", " ", text)
    text = re.sub(r"\^", " ^ ", text)
    text = re.sub(r"\+", " + ", text)
    text = re.sub(r"\-", " - ", text)
    text = re.sub(r"\=", " = ", text)
    text = re.sub(r"'", " ", text)
    text = re.sub(r"(\d+)(k)", r"\g<1>000", text)
    text = re.sub(r":", " : ", text)
    text = re.sub(r" e g ", " eg ", text)
    text = re.sub(r" b g ", " bg ", text)
    text = re.sub(r" u s ", " american ", text)
    text = re.sub(r"\0s", "0", text)
    text = re.sub(r" 9 11 ", "911", text)
    text = re.sub(r"e - mail", "email", text)
    text = re.sub(r"j k", "jk", text)
    text = re.sub(r"\s{2,}", " ", text)

    text = text.split()

    return text
  
  


In [0]:
# import json file from drive and parse in python 

import json

dict_path='/content/drive/My Drive/Major/vocabulary.json'

# read file
with open(dict_path, 'r') as my_file:
    data=my_file.read()

# parse file
vocab_dictionary = json.loads(data)

In [0]:
# sample question
ques1='What are the five different styles of questions?'
ques2='How many different types of questions are there?'

In [19]:
ques1_index=[]
for word in text_to_word_list(ques1):
  if word in vocab_dictionary:
    ques1_index.append(vocab_dictionary[word])
ques1_index

[1, 115, 3, 1466, 549, 25797, 188]

In [0]:
# function to index tokenized words using training vocabulary dictionary
def word_indexer(question):
  ques_index=[]

  for word in text_to_word_list(question):
    if word in vocab_dictionary:
      ques_index.append(vocab_dictionary[word])      
  return ques_index

In [21]:
# preprocess and tokenize sample questions 
q1=word_indexer(ques1)
print(q1)
q2=word_indexer(ques2)
print(q2)

[1, 115, 3, 1466, 549, 25797, 188]
[26, 184, 549, 547, 188, 115, 307]


In [22]:
# zero padding above tokenized indices

from tensorflow.keras.preprocessing.sequence import pad_sequences

question_list=[q1,q2]
question_list=pad_sequences(question_list,maxlen = 50)
print(question_list)

[[    0     0     0     0     0     0     0     0     0     0     0     0
      0     0     0     0     0     0     0     0     0     0     0     0
      0     0     0     0     0     0     0     0     0     0     0     0
      0     0     0     0     0     0     0     1   115     3  1466   549
  25797   188]
 [    0     0     0     0     0     0     0     0     0     0     0     0
      0     0     0     0     0     0     0     0     0     0     0     0
      0     0     0     0     0     0     0     0     0     0     0     0
      0     0     0     0     0     0     0    26   184   549   547   188
    115   307]]


In [0]:
# manhattan distance calculation function
import tensorflow.keras.backend as K
def manhattan_distance(left,right):
  return K.exp(-K.sum(K.abs(left-right),axis=1, keepdims=True))

In [24]:
# Recreate the exact same model, including weights and optimizer.
import tensorflow as tf
from tensorflow.keras.models import load_model

model_path='/content/drive/My Drive/Major/85DQD2.h5'
new_model = load_model(model_path)
new_model.summary()

Model: "model_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
left_input (InputLayer)         [(None, 50)]         0                                            
__________________________________________________________________________________________________
right_input (InputLayer)        [(None, 50)]         0                                            
__________________________________________________________________________________________________
embedding_layer (Embedding)     (None, 50, 300)      25796400    left_input[0][0]                 
                                                                 right_input[0][0]                
__________________________________________________________________________________________________
bidirectional_1 (Bidirectional) (None, 30)           38040       embedding_layer[0][0]      

In [0]:
que1=question_list[0]
que1=np.asarray(que1)
que2=question_list[1]
que2=np.asarray(que2)


In [26]:
# predict sample questions on saved model

# loss, acc = new_model.predict([question_list[0],question_list[1]])
# # print("Restored model, accuracy: {:5.2f}%".format(100*acc))

pred=new_model.predict([[que1],[que2]])
print(pred)

[[0.40954342]]
